In [ ]:
from itertools import pairwise

with open("5/input.txt") as f:
    data = f.read()
data

In [ ]:
categories = data.split("\n\n")
categories

In [ ]:
seeds = list(map(lambda seed_str: int(seed_str.strip()), categories[0].replace("seeds: ", "").split(" ")))
seeds

In [ ]:
def get_missing_ranges(category_map: list[tuple[int, int, int]]):
    missing_ranges = []
    for seq1, seq2 in pairwise(category_map):
        if seq1[1] != (seq2[0] - 1):
            source_range_start, source_range_end_inclusive, destination_range_start = seq1[1]+1, seq2[0] - 1, seq1[1]+1
            missing_ranges.append((source_range_start, source_range_end_inclusive, destination_range_start))
    return missing_ranges

In [ ]:
category_maps = {}

for category in categories[1:]:
    category_map_name, mappings_str = category.split(" map:\n", maxsplit=1)
    from_category, to_category = category_map_name.split("-to-")
    mapping_lines = mappings_str.strip().split("\n")
    from_to_category = []
    for line in mapping_lines:
        destination_range_start, source_range_start, range_length = map(int, line.split())
        source_range_end_inclusive = source_range_start + range_length - 1
        from_to_category.append((source_range_start, source_range_end_inclusive, destination_range_start))
    from_to_category = sorted(from_to_category, key=lambda x: x[0])
    missing_ranges = get_missing_ranges(from_to_category)
    from_to_category.extend(missing_ranges)
    from_to_category = sorted(from_to_category, key=lambda x: x[0])
    category_maps[from_category] = (to_category, from_to_category)
category_maps

In [ ]:
def find_matching_range_linear(value: int, from_to_map: list[tuple[int, int, int]]) -> tuple[int | None, tuple[int | None, int | None], int]:
    for i, (source_range_start, source_range_end_inclusive, destination_range_start) in enumerate(from_to_map):
        if source_range_start <= value <= source_range_end_inclusive:
            diff_from_source_start = value - source_range_start
            return i, (source_range_start, source_range_end_inclusive), (destination_range_start + diff_from_source_start)
    return value

def find_matching_range_binary(value: int, from_to_map: list[tuple[int, int, int]]) -> tuple[int | None, tuple[int, int] | tuple[None, None], int]:
    start_index, end_index = 0, len(from_to_map)

    while start_index < end_index:
        mid_index = (end_index + start_index) // 2
        source_range_start, source_range_end_inclusive, destination_range_start = from_to_map[mid_index]

        if source_range_start <= value <= source_range_end_inclusive:
            diff_from_source_start = value - source_range_start
            return mid_index, (source_range_start, source_range_end_inclusive), (destination_range_start + diff_from_source_start)
        
        elif value > source_range_end_inclusive:
            start_index = mid_index + 1

        else:
            end_index = mid_index
    return None, (None, None), value


def map_from_to_category(value: int, from_to_map: list[tuple[int, int, int]]):
    *_, mapped_value = find_matching_range_binary(value, from_to_map)
    return mapped_value

In [ ]:
def map_seeds_all_categories(start_values: list[int], category_maps: dict[str, tuple[str, list[tuple[int, int, int]]]]) -> list[int]:
    start_category = "seed"
    mapped_values = start_values
    
    while start_category in category_maps:
        start_category, category_map = category_maps[start_category]
        mapped_values = [map_from_to_category(value, category_map) for value in mapped_values]
    
    return mapped_values


In [ ]:
locations = map_seeds_all_categories(seeds, category_maps)
locations

In [ ]:
min(locations)

## Part 2

In [ ]:
seed_ranges = [(range_start, range_start + range_length - 1) for range_start, range_length in list(zip(seeds[::2], seeds[1::2]))]
seed_ranges = sorted(seed_ranges)
seed_ranges

In [ ]:
category_maps['soil'][1]

In [ ]:
def ranges_overlap(first_range_start: int, first_range_end: int, second_range_start: int, second_range_end: int) -> bool:
    return (
        (first_range_start <= second_range_start <= first_range_end) or
        (first_range_start <= second_range_end <= first_range_end) or
        (second_range_start <= first_range_start <= second_range_end) or
        (second_range_start <= first_range_end <= second_range_end)
    )

def get_overlapping_ranges(range: tuple[int, int], category_map: list[tuple[int, int, int]]) -> list[tuple[int, int, int]]:
    range_start, range_end_inclusive = range
    res = []
    for value_range in category_map:
        source_range_start, source_range_end_inclusive, _ = value_range
        if ranges_overlap(source_range_start, source_range_end_inclusive, range_start, range_end_inclusive):
            res.append(value_range)
    if len(res) == 0:
        res = [(range_start, range_end_inclusive, range_start)]
    else:
        (first_range_start, first_range_end_inclusive, first_destination_range_start), *rest = res
        if first_range_start > range_start:
            res = [(range_start, first_range_start-1, range_start), *res]

        elif first_range_start < range_start:
            destination_range_start = first_destination_range_start + (range_start - first_range_start)
            res = [(range_start, first_range_end_inclusive, destination_range_start), *rest]
        

        *rest, (last_range_start, last_range_end_inclusive, last_destination_range_start) = res
        if last_range_end_inclusive < range_end_inclusive:
            res = [*res, (last_range_end_inclusive+1, range_end_inclusive, last_range_end_inclusive+1)]

        elif last_range_end_inclusive > range_end_inclusive:
            res = [*rest, (last_range_start, range_end_inclusive, last_destination_range_start)]

    return res


def get_mapped_ranges(range: tuple[int, int], category_map: list[tuple[int, int, int]]) -> list[tuple[int, int]]:
    overlapping_ranges = get_overlapping_ranges(range, category_map)
    mapped_ranges = []

    for source_range_start, source_range_end_inclusive, destination_range_start in overlapping_ranges:
        destination_range_end = destination_range_start + (source_range_end_inclusive - source_range_start)
        mapped_ranges.append((destination_range_start, destination_range_end))
    return mapped_ranges

def map_seed_ranges(start_ranges: list[tuple[int, int]], category_maps: dict[str, tuple[str, list[tuple[int, int, int]]]]) -> list[tuple[int, int]]:
    start_category = "seed"
    mapped_ranges = start_ranges
    
    while start_category in category_maps:
        start_category, category_map = category_maps[start_category]
        mapped_ranges = [mapped_range for range in mapped_ranges for mapped_range in get_mapped_ranges(range, category_map)]
    return mapped_ranges

mapped_locations = map_seed_ranges(seed_ranges, category_maps)
min(x[0] for x in mapped_locations)